Notebook to experiment with the lifespan estimator and assist in its development.

In [9]:
from qsharp.estimator import EstimatorParams

from quantumthreattracker.lifespan_estimator.lifespan_estimator import (
    HardwareRoadmap,
    LifespanEstimator,
)

In [10]:
hardware_roadmap = HardwareRoadmap()

params_sc = EstimatorParams() # parameters for typical superconducting qubits
params_sc.qubit_params.name = "qubit_gate_ns_e3"

params_ti = EstimatorParams() # parameters for typical trapped ion qubits
params_ti.qubit_params.name = "qubit_gate_us_e4"

for i in range(10):
    params_sc.constraints.max_physical_qubits = 1000*(2**i)
    params_ti.constraints.max_physical_qubits = 100*(2**i)
    hardware_roadmap.add(timestamp=2025+2*i, estimator_params=[params_sc, params_ti])

hardware_roadmap.as_list()

[{'timestamp': 2025,
  'hardwareList': [{'estimatorParams': {'qubitParams': {'name': 'qubit_gate_ns_e3'},
     'constraints': {'maxPhysicalQubits': 1000}}},
   {'estimatorParams': {'qubitParams': {'name': 'qubit_gate_us_e4'},
     'constraints': {'maxPhysicalQubits': 100}}}]},
 {'timestamp': 2027,
  'hardwareList': [{'estimatorParams': {'qubitParams': {'name': 'qubit_gate_ns_e3'},
     'constraints': {'maxPhysicalQubits': 2000}}},
   {'estimatorParams': {'qubitParams': {'name': 'qubit_gate_us_e4'},
     'constraints': {'maxPhysicalQubits': 200}}}]},
 {'timestamp': 2029,
  'hardwareList': [{'estimatorParams': {'qubitParams': {'name': 'qubit_gate_ns_e3'},
     'constraints': {'maxPhysicalQubits': 4000}}},
   {'estimatorParams': {'qubitParams': {'name': 'qubit_gate_us_e4'},
     'constraints': {'maxPhysicalQubits': 400}}}]},
 {'timestamp': 2031,
  'hardwareList': [{'estimatorParams': {'qubitParams': {'name': 'qubit_gate_ns_e3'},
     'constraints': {'maxPhysicalQubits': 8000}}},
   {'esti

In [11]:
hardware_roadmap.save('sample_hardware_roadmap')

In [12]:
lifespan_estimator = LifespanEstimator(hardware_roadmap)

In [13]:
lifespan = lifespan_estimator.estimate_threats("RSA", 8, detail_level=1)
lifespan

{'protocol': 'RSA-8',
 'threats': [{'timestamp': 2037, 'runtime': 2508288800},
  {'timestamp': 2039, 'runtime': 501676800},
  {'timestamp': 2039, 'runtime': 1488618000000},
  {'timestamp': 2041, 'runtime': 209025200},
  {'timestamp': 2041, 'runtime': 186078600000},
  {'timestamp': 2043, 'runtime': 163008000},
  {'timestamp': 2043, 'runtime': 114105600000}]}

In [14]:
protocols = [
    {"algorithm": "RSA", "keySize": 8},
    {"algorithm": "RSA", "keySize": 16},
    {"algorithm": "RSA", "keySize": 32},
    {"algorithm": "RSA", "keySize": 64}
    ]

In [ ]:
threat_report = lifespan_estimator.generate_threat_report(protocols=protocols, detail_level=1)
threat_report

In [ ]:
lifespan_estimator.save_threat_report('sample_threat_report')